<a href="https://colab.research.google.com/github/hobezhang/NLP-based-information-retrieval-system/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers[torch]
!pip install datasets

我们首先把该用的包都引入，比如transformers。transformers是一个开源库，它里面集成了非常多的模型结构，AutoModel，AutoTokenizer是用来根据配置文件和模型路径自动加载模型的类。


In [3]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.28.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
be

In [4]:
import transformers
from transformers import AutoModel, AutoConfig,AutoTokenizer,BertForMaskedLM,BertModel
import torch



加载我们想要加载的模型，比如`bert-base-uncased`

In [ ]:
import os
os.kill(os.getpid(), 9)

In [5]:
weights_name = "bert-base-uncased"

bert = BertForMaskedLM.from_pretrained(weights_name)

bert_tokenizer = AutoTokenizer.from_pretrained(weights_name)

print(bert_tokenizer)
#     返回一个字典：
#     input_ids: list[int],
#     token_type_ids: list[int] if return_token_type_ids is True (default)
#     attention_mask: list[int] if return_attention_mask is True (default)
#     overflowing_tokens: list[int] if the tokenizer is a slow tokenize, else a List[List[int]] if a ``max_length`` is specified and ``return_overflowing_tokens=True``
#     special_tokens_mask: list[int] if ``add_special_tokens`` if set to ``True``and return_special_tokens_mask is True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


让我们来看看bert的tokenizer怎么对人类文本进行分词的吧！

---



In [6]:
example_text = "nice to meet you too, NewYork!"
bert_tokenizer.tokenize(example_text)

['nice', 'to', 'meet', 'you', 'too', ',', 'new', '##yo', '##rk', '!']

再来看看bert怎么对人类文本进行编码的。

In [7]:
ex_ids = bert_tokenizer.encode(example_text, add_special_tokens=True)

ex_ids

[101, 3835, 2000, 3113, 2017, 2205, 1010, 2047, 7677, 8024, 999, 102]

我们再把它编码的东西倒回成词表对应的词元来看看有什么不一样的变化：

In [8]:
bert_tokenizer.convert_ids_to_tokens(ex_ids)

['[CLS]',
 'nice',
 'to',
 'meet',
 'you',
 'too',
 ',',
 'new',
 '##yo',
 '##rk',
 '!',
 '[SEP]']

接下来我们根据源码来探究这些词元是怎么被处理的：

In [9]:
with torch.no_grad():
    reps = bert(torch.tensor([ex_ids]))
reps.keys()

odict_keys(['logits'])

In [10]:
print(reps)
#print(reps.last_hidden_state.shape)
print(torch.tensor([ex_ids]).shape)
#reps.pooler_output.shape
#[CLS]
#first_token_tensor = hidden_states[:, 0]

MaskedLMOutput(loss=None, logits=tensor([[[ -7.2896,  -7.2086,  -7.2195,  ...,  -6.4853,  -6.6456,  -4.2292],
         [ -9.7493,  -9.8236, -10.0082,  ...,  -8.3664,  -8.2071,  -7.1298],
         [-11.9456, -11.8921, -12.0648,  ...,  -8.7408, -10.5396,  -7.3491],
         ...,
         [ -6.7922,  -6.9147,  -6.5770,  ...,  -6.6145,  -6.8316,  -4.8200],
         [-11.4142, -11.5771, -11.9262,  ..., -11.3995, -11.2583,  -4.1430],
         [-12.5355, -12.2678, -12.0178,  ..., -10.2712,  -9.9876, -10.8529]]]), hidden_states=None, attentions=None)
torch.Size([1, 12])


接下来我们看一看如何进行bert的MLM任务

In [11]:
#首先加载数据集
# ! pip install datasets
from datasets import load_dataset

ag_news = load_dataset("ag_news",split="train[:5000]")
ag_news = ag_news.train_test_split(test_size=0.2)

In [ ]:
print(ag_news)
print(ag_news["train"][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})
{'text': 'DARPA awards robot aircraft deal Officials at the Defense Advanced Research Projects Agency (DARPA) awarded a \\$1 billion contract to an industry team led by Northrop Grumman Corp. to build three robotic reconnaissance aircraft and a computer operating system to help fly them for the Air ...', 'label': 3}


In [12]:
def preprocess_function(examples):
    return bert_tokenizer(examples["text"])

# def preprocess_function(examples,examples2):
#     return bert_tokenizer(examples["text"],examples2["text"])

In [13]:
preprocess_function(ag_news["train"][:2])

# preprocess_function(ag_news["train"][:2],ag_news["train"][2:4])

{'input_ids': [[101, 1001, 4464, 1025, 1996, 2088, 2003, 2062, 4795, 1001, 4464, 1025, 1996, 2162, 2006, 7404, 2038, 2081, 1996, 2088, 1037, 2062, 4795, 2173, 1010, 4501, 1001, 4464, 1025, 1055, 2343, 2038, 2056, 1012, 2236, 2566, 26132, 14163, 7377, 11335, 2546, 2056, 1996, 2088, 2001, 22035, 2102, 1025, 7078, 22035, 2102, 1025, 2625, 3647, 1010, 2138, 1996, 2162, 2001, 2025, 12786, 1996, 5320, 1997, 10130, 1012, 102], [101, 20014, 14663, 1018, 4160, 3279, 21255, 2015, 2006, 3715, 3137, 3193, 1010, 10250, 10128, 1012, 1006, 9706, 1007, 1011, 1011, 20014, 14663, 4297, 1012, 1001, 4464, 1025, 1055, 3279, 8723, 2005, 1996, 2959, 4284, 3092, 2251, 2861, 1010, 3480, 2011, 12430, 12348, 4341, 1997, 1996, 2194, 1001, 4464, 1025, 1055, 4171, 1998, 5446, 4007, 1998, 2019, 25172, 3715, 2013, 2049, 3247, 2000, 5271, 2028, 1997, 2049, 5661, 1010, 1996, 2194, 2056, 1012, 1012, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
tokenized_ag_news = ag_news.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=ag_news["train"].column_names,
)
tokenized_ag_news["train"][0]

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

{'input_ids': [101,
  1001,
  4464,
  1025,
  1996,
  2088,
  2003,
  2062,
  4795,
  1001,
  4464,
  1025,
  1996,
  2162,
  2006,
  7404,
  2038,
  2081,
  1996,
  2088,
  1037,
  2062,
  4795,
  2173,
  1010,
  4501,
  1001,
  4464,
  1025,
  1055,
  2343,
  2038,
  2056,
  1012,
  2236,
  2566,
  26132,
  14163,
  7377,
  11335,
  2546,
  2056,
  1996,
  2088,
  2001,
  22035,
  2102,
  1025,
  7078,
  22035,
  2102,
  1025,
  2625,
  3647,
  1010,
  2138,
  1996,
  2162,
  2001,
  2025,
  12786,
  1996,
  5320,
  1997,
  10130,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [15]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [16]:
lm_dataset = tokenized_ag_news.map(group_texts, batched=True, num_proc=4)
print(lm_dataset)
print(lm_dataset["train"][0]["input_ids"])

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1739
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 446
    })
})
[101, 1001, 4464, 1025, 1996, 2088, 2003, 2062, 4795, 1001, 4464, 1025, 1996, 2162, 2006, 7404, 2038, 2081, 1996, 2088, 1037, 2062, 4795, 2173, 1010, 4501, 1001, 4464, 1025, 1055, 2343, 2038, 2056, 1012, 2236, 2566, 26132, 14163, 7377, 11335, 2546, 2056, 1996, 2088, 2001, 22035, 2102, 1025, 7078, 22035, 2102, 1025, 2625, 3647, 1010, 2138, 1996, 2162, 2001, 2025, 12786, 1996, 5320, 1997, 10130, 1012, 102, 101, 20014, 14663, 1018, 4160, 3279, 21255, 2015, 2006, 3715, 3137, 3193, 1010, 10250, 10128, 1012, 1006, 9706, 1007, 1011, 1011, 20014, 14663, 4297, 1012, 1001, 4464, 1025, 1055, 3279, 8723, 2005, 1996, 2959, 4284, 3092, 2251, 2861, 1010, 3480, 2011, 12430, 12348, 4341, 1997, 1996, 2194, 1001, 4464, 1025, 1055, 4171, 1998, 5446, 4007,

In [17]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=bert_tokenizer, mlm_probability=0.15)
#15%采用MASK机制，其中80%用MASK替代，10%保持不变，10%用别的单词替代

torch.Size([1, 7, 30522])

nice pointless meet you.

nice [unused653] meet you.

nice social meet you.



In [18]:

from transformers import TrainingArguments, Trainer
import os
import os

training_args = TrainingArguments(
    output_dir="hello_bert",
    evaluation_strategy="epoch",
    per_device_train_batch_size =32,
    per_device_eval_batch_size =32,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=bert,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.776077
2,No log,2.619334
3,No log,2.586496


TrainOutput(global_step=165, training_loss=2.837491122159091, metrics={'train_runtime': 157.2976, 'train_samples_per_second': 33.166, 'train_steps_per_second': 1.049, 'total_flos': 343284877094400.0, 'train_loss': 2.837491122159091, 'epoch': 3.0})

In [19]:
text = "nice [MASK] meet you."
inputs = bert_tokenizer(text, return_tensors = "pt").to(bert.device)
mask_token_index = torch.where(inputs["input_ids"] == bert_tokenizer.mask_token_id)[1]
logits = bert(**inputs).logits
print(logits.shape)
mask_token_logits = logits[0, mask_token_index, :]
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

for token in top_3_tokens:
    print(text.replace(bert_tokenizer.mask_token, bert_tokenizer.decode([token])))

torch.Size([1, 7, 30522])
nice to meet you.
nice ta meet you.
nice meeting meet you.


接下来我们探索一下，使用未预训练的模型加一个分类头来微调，效果会怎么样

首先我们需要定义一个初始化的BertModel，这个Model的权重是没有经过训练更新的，也就是随机生成的权重

In [20]:
config = AutoConfig.from_pretrained("bert-base-uncased")
print(config)
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#初始化Bert，config主要是模型的框架，不包括权重
init_bert = BertModel(config)
init_bert


BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

我们先看看基础的BertModel离预测Token还有多大的差别

In [21]:
text = "nice [MASK] meet you."
inputs = bert_tokenizer(text, return_tensors = "pt").to(init_bert.device)
print(inputs)
output = init_bert(**inputs)
print(output.keys())
print(output.last_hidden_state.shape)

{'input_ids': tensor([[ 101, 3835,  103, 3113, 2017, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
odict_keys(['last_hidden_state', 'pooler_output'])
torch.Size([1, 7, 768])


想办法定义一种我们的Bert，它必须可以把隐藏信息给转换为确定的Token概率

In [22]:
import torch.nn as nn
from transformers.modeling_outputs import MaskedLMOutput
class MyBertForMLM(nn.Module):
  def __init__(self, bert, config):
    super().__init__()
    self.bert = bert
    self.config = config
    self.convert_HS2Token = nn.Linear(config.hidden_size, config.vocab_size)

  def forward(self, input_ids, labels, **kargs):
    last_hidden_state = self.bert(input_ids = input_ids).last_hidden_state
    logits = self.convert_HS2Token(last_hidden_state)
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(logits.view(-1, self.config.vocab_size), labels.view(-1))
    return MaskedLMOutput(
        loss=loss,
        logits=logits,
    )



my_bertForMLM =  MyBertForMLM(init_bert,config).to("cuda")

看一看未预训练过的模型不训练会输出一坨什么

In [23]:
inputs["labels"] = inputs["input_ids"].clone()
mask_token_index = torch.where(inputs["input_ids"] == bert_tokenizer.mask_token_id)[1]
inputs = inputs.to("cuda")
logits = my_bertForMLM(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

for token in top_3_tokens:
    print(text.replace(bert_tokenizer.mask_token, bert_tokenizer.decode([token])))

nice purpose meet you.
nice cried meet you.
nice massive meet you.


尝试训练一下，然后看看输出什么一坨什么

In [24]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="hello_bert",
    evaluation_strategy="epoch",
    per_device_train_batch_size =32,
    per_device_eval_batch_size =32,
    learning_rate=1e-4,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=my_bertForMLM,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

inputs = inputs.to(next(my_bertForMLM.parameters()).device)
logits = my_bertForMLM(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()
for token in top_3_tokens:
    print(text.replace(bert_tokenizer.mask_token, bert_tokenizer.decode([token])))

Epoch,Training Loss,Validation Loss
1,No log,7.462662
2,No log,7.301681
3,No log,7.280531
4,No log,7.323056
5,No log,7.202044


nice . meet you.
nice the meet you.
nice , meet you.


接下来我们再探索一下，用预训练好的模型，加一个分类头微调的效果

In [25]:
pretrained_bert = BertModel.from_pretrained("bert-base-uncased")
my_bertForMLM = MyBertForMLM(pretrained_bert, config).to("cuda")
inputs = inputs.to(next(my_bertForMLM.parameters()).device)
logits = my_bertForMLM(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

for token in top_3_tokens:
    print(text.replace(bert_tokenizer.mask_token, bert_tokenizer.decode([token])))

from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="hello_bert",
    evaluation_strategy="epoch",
    per_device_train_batch_size =32,
    per_device_eval_batch_size =32,
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=my_bertForMLM,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()
inputs = inputs.to(next(my_bertForMLM.parameters()).device)
logits = my_bertForMLM(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

for token in top_3_tokens:
    print(text.replace(bert_tokenizer.mask_token, bert_tokenizer.decode([token])))

nice attachment meet you.
nice ito meet you.
nice middleton meet you.


Epoch,Training Loss,Validation Loss
1,No log,9.245829
2,No log,8.662988
3,No log,8.244669
4,No log,8.036317
5,No log,7.897931


nice to meet you.
nice . meet you.
nice the meet you.
